In [4]:
from pathlib import Path

In [1]:
%cd /Users/joregan/Playing/cvpr-control

/Users/joregan/Playing/cvpr-control


In [2]:
sentences = []
with open("input_sentences.txt") as f:
    for line in f.readlines():
        sentences.append(line.strip())

In [8]:
items = {}
for tsvfile in Path("tsv").glob("**/*.tsv"):
    ident = tsvfile.stem
    if ".interloctr." in str(tsvfile):
           continue
    parts = []
    with open(tsvfile) as inf:
            for line in inf.readlines():
                    parts.append(line.strip().split("\t"))
    items[ident] = {}
    items[ident]["rec"] = " ".join([x[-1] for x in parts])
    items[ident]["original"] = sentences[int(ident.split("_")[-1])]

In [10]:
from string import punctuation

PUNCT = set(punctuation)

def clean_sentence(text):
    words = []
    text = text.replace("—", " ")
    for word in text.split(" "):
        if word.startswith("[") and word.endswith("]"):
            continue
        while word[0:1] in PUNCT:
            word = word[1:]
        while word[-1:] in PUNCT:
            word = word[:-1]
        words.append(word.lower())
    return " ".join(words)

In [18]:
def prune_fillers(text):
    FILLERS = ["uh", "um"]
    words = [x for x in text.split(" ") if x not in FILLERS]
    return " ".join(words)

In [24]:
rest = {}
for item in items:
    cleaned = clean_sentence(items[item]["original"])
    if items[item]["original"] == items[item]["rec"]:
        items[item]["match"] = "exact"
    elif cleaned == items[item]["rec"]:
        items[item]["match"] = "clean"
    elif prune_fillers(cleaned) == items[item]["rec"]:
        items[item]["match"] = "clean,fillers"
    elif prune_fillers(cleaned) == prune_fillers(items[item]["rec"]):
        items[item]["match"] = "clean,fillers_both"
    else:
        rest[item] = items[item]

In [ ]:
%pip install jiwer

In [30]:
from jiwer import wer

In [32]:
for item in items:
    if not "match" in items[item]:
        s_wer = wer(clean_sentence(items[item]["original"]), items[item]["rec"])
        items[item]["wer"] = s_wer

In [41]:
from difflib import SequenceMatcher

insertions = {}
deletions = {}
replacements = {}

for item in items:
    if not "match" in items[item]:
        sent_id = item.split("_")[-1]
        a = clean_sentence(items[item]["original"]).split(" ")
        b = items[item]["rec"].split(" ")
        s = SequenceMatcher(None, a, b)
        for tag, i1, i2, j1, j2 in s.get_opcodes():
            if tag == "equal":
                continue
            if tag == "replace":
                pair = (sent_id, " ".join(a[i1:i2]), " ".join(b[j1:j2]))
                if not pair in replacements:
                    replacements[pair] = 1
                else:
                    replacements[pair] += 1
            if tag == "delete":
                pair = (sent_id, " ".join(a[i1:i2]))
                if not pair in deletions:
                    deletions[pair] = 1
                else:
                    deletions[pair] += 1
            if tag == "insert":
                pair = (sent_id, " ".join(b[j1:j2]))
                if not pair in insertions:
                    insertions[pair] = 1
                else:
                    insertions[pair] += 1
